# Streaming Output: 노드별 스트리밍

이 노트북에서는 **그래프 실행 중 노드별로 결과를 스트리밍**하는 방법을 배웁니다.

## Streaming이란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Streaming의 개념                                │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 실행 (invoke):                                              │
│   입력 → [처리 중...] → 전체 결과 반환                             │
│   (모든 처리가 끝날 때까지 기다림)                                 │
│                                                                    │
│   스트리밍 실행 (stream):                                          │
│   입력 → 노드1 결과 → 노드2 결과 → 노드3 결과 → ...               │
│   (각 노드 완료 시마다 즉시 결과 반환)                             │
│                                                                    │
│   장점:                                                            │
│   • 실시간 진행 상황 확인                                          │
│   • 사용자 경험 향상 (응답성)                                      │
│   • 디버깅 용이                                                    │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## stream_mode 옵션

```
┌────────────────────────────────────────────────────────────────────┐
│                    stream_mode 비교                                │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   stream_mode='values' (기본):                                     │
│   → 각 노드 후 전체 State 반환                                    │
│                                                                    │
│   stream_mode='updates':                                           │
│   → 각 노드가 변경한 부분만 반환                                  │
│   → 더 효율적, 변경 사항 추적 용이                                │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph duckduckgo-search langchain-community

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 에이전트 그래프 준비

In [ ]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)


search = DuckDuckGoSearchRun()
tools = [search, calculator]
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)


class State(TypedDict):
    messages: Annotated[list, add_messages]


def model_node(state: State) -> State:
    res = model.invoke(state['messages'])
    return {'messages': res}


builder = StateGraph(State)
builder.add_node('model', model_node)
builder.add_node('tools', ToolNode(tools))
builder.add_edge(START, 'model')
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')

graph = builder.compile()

print("✅ 에이전트 그래프 준비 완료")

# 3. 기본 스트리밍 (values 모드)

In [ ]:
input_data = {
    'messages': [
        HumanMessage(
            '미국의 제30대 대통령이 사망했을 때 몇 살이었나요?'
        )
    ]
}

print("=== stream_mode='values' (기본) ===")
print("각 노드 후 전체 State 반환\n")

for i, chunk in enumerate(graph.stream(input_data)):
    print(f"--- Chunk {i+1} ---")
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        if 'messages' in node_output:
            last_msg = node_output['messages'][-1] if isinstance(node_output['messages'], list) else node_output['messages']
            content = str(last_msg.content)[:100] if hasattr(last_msg, 'content') else str(last_msg)[:100]
            print(f"내용: {content}...")
    print()

# 4. updates 모드 스트리밍

In [ ]:
print("=== stream_mode='updates' ===")
print("각 노드가 변경한 부분만 반환\n")

for i, chunk in enumerate(graph.stream(input_data, stream_mode='updates')):
    print(f"--- Update {i+1} ---")
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        print(f"변경 내용: {str(node_output)[:200]}...")
    print()

# 5. 실시간 진행 상황 표시

In [ ]:
import time

print("=== 실시간 진행 상황 ===")
print(f"질문: {input_data['messages'][0].content}\n")

start_time = time.time()

for chunk in graph.stream(input_data, stream_mode='updates'):
    elapsed = time.time() - start_time
    
    for node_name, node_output in chunk.items():
        if node_name == 'model':
            msg = node_output.get('messages', [])[-1] if node_output.get('messages') else None
            if msg and hasattr(msg, 'tool_calls') and msg.tool_calls:
                print(f"[{elapsed:.1f}s] 🔧 도구 호출 결정: {[tc['name'] for tc in msg.tool_calls]}")
            elif msg and hasattr(msg, 'content') and msg.content:
                print(f"[{elapsed:.1f}s] 💬 최종 응답 생성 중...")
        elif node_name == 'tools':
            print(f"[{elapsed:.1f}s] ⚡ 도구 실행 완료")

print(f"\n총 소요 시간: {time.time() - start_time:.1f}초")

---

## 정리: Streaming Output

### 핵심 코드

```python
# 기본 스트리밍 (전체 State)
for chunk in graph.stream(input_data):
    print(chunk)

# updates 모드 (변경 사항만)
for chunk in graph.stream(input_data, stream_mode='updates'):
    print(chunk)
```

### stream_mode 비교

| 모드 | 반환 내용 | 사용 시나리오 |
|------|----------|---------------|
| **values** | 전체 State | 전체 상태 필요 시 |
| **updates** | 변경된 부분만 | 효율적인 처리, 변경 추적 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.1).bind_tools(tools)

# 변경
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)
```

## 다음 단계

**토큰 단위 스트리밍**으로 더 세밀한 실시간 출력을 구현합니다. (04번 노트북)